In [1]:
import random
import numpy as np
import copy
# from games import *
random.seed(109)

## Writeup:

I added the utility and actions methods to help with the implementation of the minimax algorithm, then implemented a function to simulate a random player versus a random player where the random player that goes first wins about 56 percent of the time if ties aren't considered. Then I implemented a minimax search algorithm based on the algorithm from the aima library but adapted to fit the homework 7 implementation of the Mancala class, and the depth to which the minimax function searches can be adjusted. My main issue so far is that my minimax function is extremely slow and simulating 100 games at a depth of 5 plies takes hours. I suspect this has to do with the fact that my minimax implementation uses copy.deepcopy() to create a new class object for every leaf in the recursively defined tree. To fix this, I will try to alter my mancala class implementation so that I can pass individual states into its methods rather than having the game state stored in the object itself. That way, I can construct the minimax nodes without having to create new mancala objects.

In [14]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        if pit < 1 or pit > self.pits_per_player:
            return False
        if self.current_player == 1:
            if self.board[self.p1_pits_index[0] + (pit - 1)] < 1:
                return False
        if self.current_player == 2:
            if self.board[self.p2_pits_index[0] + (pit - 1)] < 1:
                return False
        return True
        
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_1_pits_indices = list(range(self.p1_pits_index[0], self.p1_pits_index[1]+1))
        player_2_pits_indices = list(range(self.p2_pits_index[0], self.p2_pits_index[1]+1))

        
        if self.current_player == 1:
            player_1_available = list(filter(lambda x: player_1_pits[x] > 0, range(self.pits_per_player)))
            rand_move = random.choice(player_1_available) + 1

        else:
            player_2_available = list(filter(lambda x: player_2_pits[x] > 0, range(self.pits_per_player)))
            rand_move = random.choice(player_2_available) + 1


        self.play(rand_move)

        return self.board
    
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        if not self.valid_move(pit):
            # print("INVALID MOVE")
            return self.board
        
        if self.winning_eval():
            # print("GAME OVER")
            return self.board

        player_1_pits_indices = range(self.p1_pits_index[0], self.p1_pits_index[1]+1)
        player_2_pits_indices = range(self.p2_pits_index[0], self.p2_pits_index[1]+1)

        self.moves.append((self.current_player, pit))
        
        if self.current_player == 1:
            index = self.p1_pits_index[0] + (pit - 1)
            num_stones = self.board[index]
            self.board[index] = 0
            next_open = index + 1
            for i in range(num_stones):
                if next_open >= len(self.board) - 1:
                    next_open = 0
                self.board[next_open] += 1
                next_open += 1
            last_placed = next_open - 1
            if last_placed in player_1_pits_indices:
                # opposite = (self.pits_per_player - last_placed) * 2 + 2
                opposite = self.p2_pits_index[0] + (self.pits_per_player - (last_placed + 1))
                if self.board[opposite] > 0 and self.board[last_placed] == 1:
                    self.current_player = 1
                    return self.board
            self.current_player = 2
            return self.board

        index = self.p2_pits_index[0] + (pit - 1)
        num_stones = self.board[index]
        self.board[index] = 0
        next_open = index + 1
        for i in range(num_stones):
            if next_open >= len(self.board):
                next_open = 0
            if next_open == self.p1_pits_index[1] + 1:
                next_open += 1
            self.board[next_open] += 1
            next_open += 1
        last_placed = next_open - 1
        if last_placed in player_2_pits_indices:
            opposite = self.p1_pits_index[0] + (self.pits_per_player - (last_placed + 1))
            if self.board[opposite] > 0 and self.board[last_placed] == 1:
                self.current_player = 2
                return self.board
        self.current_player = 1
        return self.board

        
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_mancala = self.board[self.p2_mancala_index]
        

        if sum(player_1_pits) == 0 or sum(player_2_pits) == 0:
            if player_1_mancala > player_2_mancala:
                return 1
            elif player_2_mancala > player_1_mancala:
                return 2
            else:
                return 3

        return False

    def utility(self):
        return self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]

    def actions(self):
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        
        if self.current_player == 1:
            player_1_available = list(filter(lambda x: player_1_pits[x] > 0, range(self.pits_per_player)))
            moves_available = [i + 1 for i in player_1_available]
            return moves_available

        else:
            player_2_available = list(filter(lambda x: player_2_pits[x] > 0, range(self.pits_per_player)))
            moves_available = [i + 1 for i in player_2_available]
            return moves_available

In [3]:
def random_game():    
    game = Mancala()
    while not game.winning_eval():
            game.random_move_generator()
            # game.display_board()
        
    return game.winning_eval()

winning_list = np.array([random_game() for i in range(1000)])

winning_list_filter = winning_list < 3

winning_list = winning_list[winning_list_filter]

p1_winrate = sum((winning_list == 1)) / len(winning_list)

print("Player 1 wr in random vs random: ", p1_winrate)

Player 1 wr in random vs random:  0.5446428571428571


In [15]:
# def minmax_decision(state, game):
#     """Given a state in a game, calculate the best move by searching
#     forward all the way to the terminal states. [Figure 5.3]"""

#     player = game.to_move(state)

#     def max_value(state):
#         if game.terminal_test(state):
#             return game.utility(state, player)
#         v = -np.inf
#         for a in game.actions(state):
#             v = max(v, min_value(game.result(state, a)))
#         return v

#     def min_value(state):
#         if game.terminal_test(state):
#             return game.utility(state, player)
#         v = np.inf
#         for a in game.actions(state):
#             v = min(v, max_value(game.result(state, a)))
#         return v

#     # Body of minmax_decision:
#     return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))

def minmax_tree(game, plies):
    depth = 0
    
    def max_value(game, depth):
        if game.winning_eval() or depth > plies:
            return game.utility()
        v = -np.inf
        for action in game.actions():
            newgame = copy.deepcopy(game)
            newgame.play(action)
            v = max(v, min_value(newgame, depth + 1))
        return v

    def min_value(game, depth):
        if game.winning_eval() or depth > plies:
            return game.utility()
        v = np.inf
        for action in game.actions():
            newgame = copy.deepcopy(game)
            newgame.play(action)
            v = min(v, max_value(newgame, depth + 1))
        return v

    games = []
    for action in game.actions():
        newgame = copy.deepcopy(game)
        newgame.play(action)
        games.append(newgame)
    
    return max(enumerate(game.actions()), key=lambda a: min_value(games[a[0]], depth))

game = Mancala()

newgame = copy.deepcopy(game)

print(minmax_tree(newgame, 5))

(2, 3)


In [19]:
def ai_game(depth):    
    game = Mancala()
    while not game.winning_eval():
            if game.current_player == 1:
                tempgame = copy.deepcopy(game)
                move = minmax_tree(tempgame, depth)[1]
                game.play(move)
            else:
                game.random_move_generator()
            # game.display_board()
    # print("GAME OVER")                      
    return game.winning_eval()

def ai_v_ai():
    game = Mancala()
    while not game.winning_eval():
        tempgame = copy.deepcopy(game)
        move = minmax_tree(tempgame, 5)[1]
        game.play(move)
        game.display_board()
    # print("GAME OVER")
    return game.winning_eval()

# depth = 2

# winning_list = np.array([ai_game(depth) for i in range(100)])

# winning_list_filter = winning_list < 3

# p1_winrate_ties = sum((winning_list == 1)) / len(winning_list)

# print("Player 1 wr in ai vs random with ties: ", p1_winrate_ties)

# winning_list = winning_list[winning_list_filter]

# p1_winrate = sum((winning_list == 1)) / len(winning_list)

# print("Player 1 wr in ai vs random with depth ", depth, ": ", p1_winrate)

def num_games(num, depth):
    for i in range(num):
        ai_game(depth)

%timeit num_games(100, 5)

KeyboardInterrupt: 